In [67]:
# install and import libraries 

import sys
import os
import json
sys.path.append(os.path.join(os.getcwd(),'..'))

import watson_developer_cloud
from watson_developer_cloud.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions

import pandas as pd

In [68]:
# create a variable with my username and password to connect to NLU

nlu = watson_developer_cloud.NaturalLanguageUnderstandingV1(version='2018-03-16',
                                                            url='https://gateway.watsonplatform.net/natural-language-understanding/api',
                                                            iam_apikey='1UMNpcS_UCpnkerMIgmX2ANLnxFtoQwuvyNtE5YHvEFN')

In [69]:
# import the dataframe from youtube (all metadata included)

#df = pd.read_csv("/Users/madelinecraft/Desktop/Classes/Computational SS/Final Project/AllComments.csv", encoding='utf-8')

#df = pd.read_json("/Users/madelinecraft/Desktop/YouTube Comments/data.json", encoding='utf-8')

#Added by Taz, 5:19 5/9
#f = open("/Users/madelinecraft/Desktop/YouTube Comments/data.json", encoding='utf-8')
#for line in f:
#    print(line)
df = pd.read_json("/Users/madelinecraft/Desktop/YouTube Comments/data.json", lines=True, encoding='utf-8')
print(df)
#with open("/Users/madelinecraft/Desktop/YouTube Comments/data.json", encoding="utf8") as f:
#    data = f.readlines()
#    data = [json.loads(line) for line in data] #convert string to dict format
#df = pd.read_json(data) #load into dataframe
#f = open("/Users/madelinecraft/Desktop/YouTube Comments/data.json", 'r')
#for line in f:
#    store_line = line[:-1]
#    df = pd.read_json(store_line)
    
#print(df)



    VideoCategoryId  VideoCommentCount  \
0                24              26111   
1                24              26111   
2                24              26111   
3                24              26111   
4                24              26111   
5                24               6119   
6                24               6119   
7                24               6119   
8                24               6119   
9                24               6119   
10               17               3702   
11               17               3702   
12               17               3702   
13               17               3702   
14               17               3702   
15               10              22850   
16               10              22850   
17               10              22850   
18               10              22850   
19               10              22850   
20               24               8550   
21               24               8550   
22               24               

In [70]:
# create a list of the text I will analyze. "commentText" is the name of the column in my dataframe

#youtube_comment_text = list(df.loc[:,"commentText"])
youtube_comment_text = list(df.loc[:,"text"])

In [71]:
# create a for loop to send every comment to NLU, asking for sentiment and emotion analysis

# When NLU doesn't recognize the language, the loop stops with an error.
# Therefore, I need to use a try and except routine so the code won't stop. 
# Everytime it stopped in the past I deleted the tweet, then I restarted the for
# loop where it stopped so I didn't need to start all over again
s_analy = []
for i in range (0,len(youtube_comment_text)): # len(youtube_comment_text)
    response = nlu.analyze(
        text=youtube_comment_text[i],
        features=Features(
            sentiment=SentimentOptions(),
            emotion=EmotionOptions()),
        language='en').get_result()
    # print('TEST')
    # print(i)
    s_analy.append(response.copy()) # s_analy is supposed to be a dictionary with nested information

In [72]:
len(s_analy)

25

In [73]:
# Sometimes I don't know why NLU calculates only sentiment, not emotions. It happens when a variable called language
# in the dictionary is different from "en" (english). It happened to me like five times, so I cleaned those tweets 
# with these lines.

no_english = []

for i in range(0, len(s_analy)):
    if (s_analy[i]['language']!= 'en'):
        no_english.append(i)

In [74]:
sent_score = len(s_analy)*[0]
sent_label = len(s_analy)*[0]

# obtain the sentiment label and score from s_analy dictionary
for i in range(0,len(s_analy)):
    sent_score[i]=s_analy[i]['sentiment']['document']['score']
    sent_label[i]=s_analy[i]['sentiment']['document']['label']

In [75]:
# add two columns to my dataframe with sentiment label and score
df['sent_score'] = sent_score
df['sent_label'] = sent_label

In [76]:
# obtain emotions values from s_analy dictionary

sadness = len(s_analy)*[0]
joy = len(s_analy)*[0]
fear = len(s_analy)*[0]
disgust = len(s_analy)*[0]
anger = len(s_analy)*[0]

for i in range(0,len(s_analy)):
    sadness[i]= s_analy[i]['emotion']['document']['emotion']['sadness']
    joy[i]= s_analy[i]['emotion']['document']['emotion']['joy']
    fear[i]= s_analy[i]['emotion']['document']['emotion']['fear']
    disgust[i]= s_analy[i]['emotion']['document']['emotion']['disgust']
    anger[i]= s_analy[i]['emotion']['document']['emotion']['anger']

In [77]:
# add five columns to my dataframe with emotions values

df['sadness'] = sadness
df['joy'] = joy
df['fear'] = fear
df['disgust'] = disgust
df['anger'] = anger

In [78]:
# create a csv with my dataframe
df.to_csv('DDBB_sent_emo.csv', sep=',', encoding='utf-8')